# Trading indicators in python

In [1]:
import pandas as pd

## Relative Strength Index

In [2]:
def get_relative_strength_index(df: pd.DataFrame, column: str, periods: int) -> pd.DataFrame:
    """ Returns the Relative Strength Index for a given column in a dataframe.
    The relative strength index is based on the observation that price 
    movements tend to be confined between upper and lower extremes. The index 
    shows whether these price movements are strong or weak.
    
    Parameters
    ----------
    df: The dataframe containing the column.
    column: The column to be used.
    period: The period to be used.
    
    Returns
    -------
    : The dataframe with the Bollinger Bands columns.
    """
    close_delta = df[column].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rs = ma_up / ma_down
    df['{}_rsi'.format(column)] = 100 - (100/(1 + rs))
    
    return df

## Bollinger Bands

In [3]:
def get_bollinger_bands(df: pd.DataFrame, column: str, period: int,
                        std_dev: float) -> pd.DataFrame:
    """ Calculates the Bollinger Bands for a given column in a dataframe.
    Bollinger bands are a technical analysis indicator that can be used to 
    identify when securities are overbought or oversold. For example, when the 
    price is approaching the upper band — it deviates from its standard value
    and therefore is closer to being overbought. However, the opposite is true
    for the direction down.
    
    Parameters
    ----------
    df: The dataframe containing the column.
    column: The column to be used.
    period: The period to be used.

    Returns
    -------
    : The dataframe with the Bollinger Bands columns.
    """

    df['{}_upper_band'.format(column)] = df[column].rolling(period).mean() + (df[column].rolling(period).std() * std_dev)
    df['{}_lower_band'.format(column)] = df[column].rolling(period).mean() - (df[column].rolling(period).std() * std_dev)
    return df

##  Moving average convergence divergence (MACD)

In [4]:
def get_macd(df: pd.DataFrame, column: str, period1: int, period2: int,
             period3: int) -> pd.DataFrame:
    """ Calculates the MACD for a given column in a dataframe.
    Moving average convergence divergence (MACD) is a technical analysis 
    indicator that measures the difference between a security’s 26-period and 
    12-period moving averages. MACD is calculated by subtracting the 26-period 
    moving average from the 12-period moving average. If MACD is positive, it 
    indicates that the 12-period moving average is above the 26-period moving 
    average, and vice versa.
    
    Parameters
    ----------
    df: The dataframe containing the column.
    column: The column to be used.
    period1: The first period to be used.
    period2: The second period to be used.
    period3: The third period to be used.
    
    Returns
    -------
    : The dataframe with the  MACD columns.
    """

    df['{}_macd'.format(column)] = df[column].ewm(span=period1).mean() - df[column].ewm(span=period2).mean()
    df['{}_signal'.format(column)] = df['{}_macd'.format(column)].ewm(span=period3).mean()
    return df

## Return Percentage

In [5]:
def calculate_return_percentage(starting_value: float, ending_value: float,
                                period: int) -> float:
    """ Calculates the return percentage over period.
    Gives a reasonable estimation of the profitability of an asset.
    
    Parameters
    ----------
    starting_value: The starting value.
    ending_value: The ending value.
    period: The period.
    
    Returns
    -------
    : The annual return percentage.
    """

    if period <= 0:
        raise ValueError("Period must be greater than 0")

    return_value = (ending_value - starting_value) / starting_value
    return (1 + return_value) ** (1 / period) - 1

## Logarithmic Returns

In [6]:
def get_log_returns(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """ Calculates the log returns for a given column in a dataframe.
    Logarithmic returns are a critical aspect of financial planning. They 
    represent the relationship between investment growth and time. This method 
    is also a frequent “building block” for other indicators as you see down 
    the line.

    Parameters
    ----------
    df: The dataframe containing the column.
    column: The column to be used.

    Returns
    -------
    : The series with the log returns column.
    """
    returns = np.log(df[column]/df[column].shift(1))
    returns.fillna(0, inplace=True)
    
    return returns

## Volatility

In [7]:
def get_volatility(df: pd.DataFrame, periods: int, column: str) -> pd.DataFrame:
    """ Calculates the volatility for a given column in a dataframe.
    Volatility is the term used to describe the fluctuations in the price of a 
    security. It helps estimate the stability of the price and therefore risks 
    related to investment in the asset.
    
    Parameters
    ----------
    df: The dataframe containing the column.
    periods: The periods to be used.
    column: The column to be used.
 
    Returns
    -------
    : The volatility series column and mean volatility.
    """

    returns = get_log_returns(df, column)
    volatility_series = returns.rolling(window=periods).std() * np.sqrt(periods)

    return volatility_series, volatility_series.mean()

## Sharpe Ratio

In [8]:
def get_sharpe_ratio(df: pd.DataFrame, periods: int, column: str) -> pd.DataFrame:
    """ Calculates the Sharpe ratio for a given column in a dataframe.
    Sharpe Ratio is an indicator that represents the spread of risks across 
    periods by measuring the relations between returns and volatility risks 
    during the designated period.

    Parameters
    ----------
    df: The dataframe containing the column.
    periods: The periods to be used.
    column: The column to be used.
    
    Returns
    -------
    : The Sharpe ratio.
    """

    returns = get_log_returns(df, column)
    volatility = get_volatility(df, periods, column)[0]

    return returns.mean()/volatility